# Preprocessing Data

### Dealing with categorical features
* Scikit-learn will not accept categorical features by default
* Need to encode categorical features numerically
* Convert to ‘dummy variables’
    * 0: Observation was NOT that category
    * 1: Observation was that category
    
### Dealing with categorical features in Python
* scikit-learn: OneHotEncoder()
* pandas: get_dummies()    

In [1]:
import pandas as pd
df = pd.read_csv('Auto.csv')
names=['mpg',
       'displacement',
       'horsepower',
       'weight',
       'acceleration',
       'origin']
#df = df[names] # only get the names that are in the example used
#df.dtypes
df.head()

FileNotFoundError: File b'Auto.csv' does not exist

In [ ]:
df_origin = pd.get_dummies(df.drop('Model', axis=1))
print(df_origin.head())

We will drop the _Origin_Japanese_ column since it gives redondent information (i.e. if it's not American or European it has to be Japanese)

In [ ]:
df_origin = df_origin.drop('Origin_Japanese', axis=1)
print(df_origin.head())

* mpg: Target Variable
* Origin: Categorical Feature

In [ ]:
X = df_origin[['Origin_American','Origin_European']].values # features

In [ ]:
y = df_origin[['MPG']].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
ridge = Ridge(alpha=0.5, normalize=True).fit(X_train,y_train)
ridge.score(X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
# Create a boxplot of MPG per Origin
df.boxplot('MPG', 'Origin', rot=60)
# Show the plot
# plt.show()

## Handling missing data

One way is to replace the missing data on each column with, for example, a _np.nan_ using _df.insulin.replace(0, np.nan, inplace=True)_. And then dropping all the rows that have NANs using _df.dropna()_. The downside of this method is the fact that a lot of data can be dropped.

### Imputing missing data
* Making an educated guess about the missing values
    * Example: Using the mean of the non-missing entries

__Option1:__
``` python
from sklearn.preprocessing import Imputer  
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)  
imp.fit(X)  
X = imp.transform(X)  
```

__Option 2:__  
``` python
from sklearn.pipeline import Pipeline  
from sklearn.preprocessing import Imputer  
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)  
logreg = LogisticRegression()  
steps = [('imputation', imp),('logistic_regression', logreg)]  
pipeline = Pipeline(steps)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)  
pipeline.fit(X_train, y_train)  
y_pred = pipeline.predict(X_test)  
pipeline.score(X_test, y_test)  
```

The last step on a pipeline must me an estimator (classifier or regressior).

## Exercise example

### Drop missing values
``` python
# Convert '?' to NaN
df[df == '?'] = np.nan

# Print the number of NaNs
print(df.isnull().sum())

# Print shape of original DataFrame
print("Shape of Original DataFrame: {}".format(df.shape))

# Drop missing values and print shape of new DataFrame
df = df.dropna()

# Print shape of new DataFrame
print("Shape of DataFrame After Dropping All Rows with Missing Values: {}".format(df.shape))
```

## Centering and Scaling Data

### Why scale your data?
* Many models use some form of distance to inform them
* Features on larger scales can unduly influence the model
* Example: k-NN uses distance explicitly when making predictions
* We want features to be on a similar scale
* Normalizing (or scaling and centering)

### Ways to normalize your data
* Standardization: Subtract the mean and divide by variance
    * All features are centered around zero and have variance one
* Can also subtract the minimum and divide by the range
    * Minimum zero and maximum one
* Can also normalize so the data ranges from -1 to +1
* See scikit-learn docs for further details

``` python
from sklearn.preprocessing import scale
In [3]: X_scaled = scale(X)
In [4]: np.mean(X), np.std(X)
Out[4]: (8.13421922452, 16.7265339794)
In [5]: np.mean(X_scaled), np.std(X_scaled)
Out[5]: (2.54662653149e-15, 1.0)
```

### Scaling in a pipeline
``` python 
from sklearn.preprocessing import StandardScaler
In [7]: steps = [('scaler', StandardScaler()),('knn', KNeighborsClassifier())]
In [8]: pipeline = Pipeline(steps)
In [9]: X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)
In [10]: knn_scaled = pipeline.fit(X_train, y_train)
In [11]: y_pred = pipeline.predict(X_test)
In [12]: accuracy_score(y_test, y_pred)
Out[12]: 0.956
In [13]: knn_unscaled = KNeighborsClassifier().fit(X_train, y_train)
In [14]: knn_unscaled.score(X_test, y_test)
Out[14]: 0.928
```

### CV and scaling in a pipeline
``` python 
In [14]: steps = [('scaler', StandardScaler()),(('knn', KNeighborsClassifier())]
In [15]: pipeline = Pipeline(steps)
In [16]: parameters = {knn__n_neighbors=np.arange(1, 50)}
In [17]: X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)
In [18]: cv = GridSearchCV(pipeline, param_grid=parameters)
In [19]: cv.fit(X_train, y_train)
In [20]: y_pred = cv.predict(X_test)
In [21]: print(cv.best_params_)
In [22]: print(cv.score(X_test, y_test))
In [23]: print(classification_report(y_test, y_pred))
```